# 需要的包

In [48]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #所有的变量信息都可以打印出来

# 数据处理


## 数据读取

In [49]:
types={'CUPS_CARD_IN': str,'SPEC_DISC_LVL': str,'TRANS_CHNL': str,'SPEC_DISC_TP': str,'CARD_BIN': str,'MCHNT_TP': str}
train = pd.read_csv('./data2/trans_dtl_train1.csv',dtype=types)
test  = pd.read_csv('./data2/trans_dtl_test1.csv',dtype=types)
target = pd.read_csv('./data2/bind_cards1.csv',header=None)
target.columns = ['encrypt_card_no']
target['target'] = '1'
full_data = [train, test]
all =pd.concat(full_data)
train.info()
train.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312647 entries, 0 to 312646
Data columns (total 32 columns):
SETTLE_DT           312647 non-null int64
encrypt_mchnt_cd    312647 non-null object
TERM_ID             310599 non-null object
MCHNT_TP            311750 non-null object
CONN_IN             312647 non-null int64
LOC_TRANS_TM        312647 non-null int64
STI_TAKEOUT_IN      312647 non-null int64
TRANS_ST            312647 non-null int64
CARD_BIN            312642 non-null object
CARD_PROD           312647 non-null int64
CARD_ATTR           312647 non-null int64
CARD_MEDIA          312647 non-null int64
CARD_BRAND          312647 non-null int64
CARD_RANK           312647 non-null int64
ISS_INS_ID_CD       312647 non-null int64
ACQ_INS_ID_CD       312647 non-null int64
TRANS_ID            312647 non-null object
TRANS_CHNL          312642 non-null object
TRANS_AT            312647 non-null int64
RESP_CD1            312311 non-null object
RESP_CD3            311665 non-null object

,SETTLE_DT,encrypt_mchnt_cd,TERM_ID,MCHNT_TP,CONN_IN,LOC_TRANS_TM,STI_TAKEOUT_IN,TRANS_ST,CARD_BIN,CARD_PROD,...,SPEC_DISC_TP,SPEC_DISC_LVL,encrypt_card_no,CUPS_CARD_IN,TRANS_MD,HCE_PROD_NM,TOUCH_TP,HCE_PROD_IN,NOPWD_PETTY_IN,WHITE_MCHNT_IN
0,20180612,42fc2d6c76d2048c9aded5dbfcd1a9e2,12042365,8220,1,72535,1,1,19620516,0,...,00,0,7ccca9ac2b0fa257c3f78e4a0a30730c,2,1,0,2,0,0,1
1,20180603,f7fcc1f8027ff424327f7f6a2a2da256,01080209,4816,1,175047,1,1,1962156831,3,...,03,0,82c32cbb47703f4c3e6fd4585bbab5ce,2,2,0,99,0,0,0
2,20180511,e552f7d0516b669e04cbecc16b0b6640,45880776,7011,0,73149,1,1,16628268,1,...,00,0,edd137b31c3112c69c5e9f7d366dd81d,2,2,0,2,0,0,0
3,20180602,047e9e99f210f924ba30260eab354531,10000001,8398,1,63544,1,1,1962309101,0,...,00,0,55342a18c527b31709f9ee9ba2663c59,2,2,Y,99,0,0,1
4,20180606,42fc2d6c76d2048c9aded5dbfcd1a9e2,12042426,8220,1,124212,1,1,19620516,0,...,00,0,953664ea99b5b2b3fddb08732a358aa7,2,1,0,2,0,0,1


In [59]:
train['MCHNT_TP'].nunique()

282

In [54]:
train['TOUCH_TP'].nunique()
train['CARD_BIN'].nunique()

3

975

## 数据统计

In [55]:
test.shape
train.shape
target['encrypt_card_no'].nunique()
test['encrypt_card_no'].nunique()
train['encrypt_card_no'].nunique()
all['encrypt_card_no'].nunique()

(1026305, 32)

(312647, 32)

1199

35001

11200

46201

## 提取简易特征 


### 数字特征

In [56]:
train['SETTLE_DT1'] =pd.to_datetime(train['SETTLE_DT'], format='%Y%m%d')
train['date_diff'] = (pd.Timestamp('2018-08-03')- train['SETTLE_DT1']).dt.days
train['TRANS_AT2'] = np.log(train['TRANS_AT'] )

In [60]:
LOC_TRANS_TM 

30

15

4

282

### 类别特征

#### MCC之下，商户类型又分为了3个层次的不同类别

In [77]:
mcc = pd.read_csv('./data2/Para/PARA_DIM_MCC.txt',encoding='gb2312',header=0)
train=pd.merge(train,mcc[['MCHNT_TP','MCC_DESC','MCC_GRP_DESC1','MCC_GRP_DESC2','MCC_GRP_DESC3']]  , on='MCHNT_TP', how='left')

In [65]:
mcc['MCC_GRP_DESC1'].nunique()
mcc['MCC_GRP_DESC2'].nunique()
mcc['MCC_GRP_DESC3'].nunique()
train['MCHNT_TP'].nunique()

30

15

4

282

#### CARD_BRAND 后续将visa/jcb/master等归为一类。

In [75]:
CARD_BRAND = pd.read_csv('./data2/Para/PARA_CARD_BRAND.txt',encoding='gb2312',header=0)
CARD_BRAND

,CARD_BRAND,CARD_BRAND_NM,REC_UPD_TS
0,0,其它,2008-11-14-13.31.18.000000
1,1,6字头银标,2008-11-14-13.31.18.000000
2,2,6字头非标,2008-11-14-13.31.18.000000
3,3,银联9字头,2008-11-14-13.31.18.000000
4,4,VISA卡,2008-11-14-13.31.18.000000
5,5,MASTER卡,2008-11-14-13.31.18.000000
6,6,JCB,2008-11-14-13.31.18.000000
7,7,美运卡,2008-11-14-13.31.18.000000
8,8,其它卡BIN,2008-11-14-13.31.18.000000


#### DIM_TRANS

In [90]:
DIM_TRANS = pd.read_csv('./data2/Para/PARA_DIM_TRANS.txt',encoding='gb2312',header=0)
DIM_TRANS

,TRANS_ID,TRANS_DESC,SETTLE_FLAG,TRANS_GRP_ID1,TRANS_GRP_DESC1,TRANS_GRP_ID2,TRANS_GRP_DESC2,TRANS_GRP_ID3,TRANS_GRP_DESC3,REC_UPD_TS
0,0,未知,1,0,其它,0,其它,0,其它,20090101
1,E32,差错-贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
2,E33,差错-一般转帐转入贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
3,E34,差错-一般转帐转出贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
4,E62,差错-贷记调整撤销,0,123,贷记调整,17,差错调帐类,2,差错类交易代码,20110419
5,E63,差错-PBOC转帐圈存贷记调整（转入）,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
6,E64,差错-PBOC转帐圈存贷记调整（转出）,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
7,E65,差错-PBOC现金充值贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
8,E66,差错-PBOC圈存贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
9,E80,差错-发卡-存款的贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101


In [87]:
# train=pd.merge(train,mcc[['MCHNT_TP','MCC_DESC','MCC_GRP_DESC1','MCC_GRP_DESC2','MCC_GRP_DESC3']]  , on='MCHNT_TP', how='left')
DIM_TRANS['TRANS_DESC'].nunique()
DIM_TRANS['TRANS_GRP_DESC1'].nunique()
DIM_TRANS['TRANS_GRP_DESC2'].nunique()
DIM_TRANS['TRANS_GRP_DESC3'].nunique()
train['TRANS_ID'].nunique()

,TRANS_ID,TRANS_DESC,SETTLE_FLAG,TRANS_GRP_ID1,TRANS_GRP_DESC1,TRANS_GRP_ID2,TRANS_GRP_DESC2,TRANS_GRP_ID3,TRANS_GRP_DESC3,REC_UPD_TS
0,0,未知,1,0,其它,0,其它,0,其它,20090101
1,E32,差错-贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
2,E33,差错-一般转帐转入贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
3,E34,差错-一般转帐转出贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
4,E62,差错-贷记调整撤销,0,123,贷记调整,17,差错调帐类,2,差错类交易代码,20110419
5,E63,差错-PBOC转帐圈存贷记调整（转入）,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
6,E64,差错-PBOC转帐圈存贷记调整（转出）,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
7,E65,差错-PBOC现金充值贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
8,E66,差错-PBOC圈存贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101
9,E80,差错-发卡-存款的贷记调整,1,123,贷记调整,17,差错调帐类,2,差错类交易代码,20090101


455

79

21

6

58

In [88]:
train=pd.merge(train,DIM_TRANS[['TRANS_ID','TRANS_GRP_DESC1','TRANS_GRP_DESC2','TRANS_GRP_DESC2']]  , on='TRANS_ID', how='left')

### 根据卡号进行聚合

In [37]:
all_together =(train.groupby(['encrypt_card_no'])
                  .agg({'TRANS_AT2': ['sum', 'mean', 'min', 'max','count'],'HCE_PROD_NM': ['min', 'max']}))


### rename column

In [38]:
all_together.columns = ["_".join(x) for x in all_together.columns.ravel()]
all_together=all_together.reset_index(level=['encrypt_card_no'],drop=False)
all_together.head(2)

,encrypt_card_no,TRANS_AT2_sum,TRANS_AT2_mean,TRANS_AT2_min,TRANS_AT2_max,TRANS_AT2_count,HCE_PROD_NM_min,HCE_PROD_NM_max
0,00050a15df80a709d3e4973543b27c7a,171.341,10.709,7.762,14.221,16,0,0
1,0007358abae98a7836c3ec09c3b51e61,-inf,-inf,-inf,10.820,17,0,0


In [39]:
all_together.shape

(11200, 8)

## 打标签


###  直接打标签

根据bind_cards1.csv打标签，绑定云闪付的为1，不绑定的为0

In [41]:
train_v=pd.merge(all_together, target, on='encrypt_card_no', how='outer')
train_v.fillna(value=0)
train_v.shape

,encrypt_card_no,TRANS_AT2_sum,TRANS_AT2_mean,TRANS_AT2_min,TRANS_AT2_max,TRANS_AT2_count,HCE_PROD_NM_min,HCE_PROD_NM_max,target
0,00050a15df80a709d3e4973543b27c7a,171.341,10.709,7.762,14.221,16,0,0,0
1,0007358abae98a7836c3ec09c3b51e61,-inf,-inf,-inf,10.820,17,0,0,0
2,0007d0ed3744d407c461ad52f60ecaa5,-inf,-inf,-inf,11.002,10,0,0,0
3,0020d72758cdcca8f26da50311342ca8,-inf,-inf,-inf,13.556,4,0,0,0
4,002127f76a64d304e9f19a85038bce4d,12.345,12.345,12.345,12.345,1,0,0,0
5,0025ee4190b5ab5fd5c03e2f05b0ab61,-inf,-inf,-inf,-inf,1,0,0,0
6,0027ac1b5fa9e2dafda079786de87806,117.155,13.017,12.371,13.476,9,0,0,0
7,002b207ef6dd6243c1b7ef658991b66b,76.662,10.952,10.102,11.930,7,0,0,0
8,002b918b3ddd263f292732f80897f36d,198.383,9.919,8.517,13.251,20,0,0,0
9,002fa8d7f3b253ef1386f87b73118b75,-inf,-inf,-inf,15.520,37,0,0,1


(11200, 9)

In [9]:
train_v.query('HCE_PROD_NM_max != "0" and target == "1"' )

,encrypt_card_no,TRANS_AT_sum,TRANS_AT_mean,TRANS_AT_min,TRANS_AT_max,TRANS_AT_count,HCE_PROD_NM_min,HCE_PROD_NM_max,target
77,01cc8ed4c6932aa05e9ea295998b3b67,29494100,116118.504,0,1988800,254,0,Z,1
162,03de3de5f9ee59460a9079e4ebc1630a,207535,15964.231,0,206000,13,0,Y,1
376,090b11b3a189f1f904726ed6853edafc,700,350.000,200,500,2,Y,Y,1
485,0b66970e2492c815b7b22da10a969df9,135300,19328.571,0,66100,7,0,Y,1
494,0ba55a8be080f6c237ba12d3d65c2b08,6301,1260.200,1,2100,5,Y,Z,1
506,0c05ee267edabae6c9fd5239b218cd71,90647,5665.438,0,61000,16,0,Y,1
580,0d7f1a2a48da25c7010bd920f63cf2f6,2966393,82399.806,0,500000,36,0,Y,1
663,0f7a2ff00c4db88b4381a1f22974b6d6,7259527,82494.625,0,2125903,88,0,Z,1
750,11b67b55d0e68a56c5802e9432574c9a,5099,1699.667,200,4699,3,1,1,1
851,13b346ffa087a1aeafff55f51e39244f,1722599,246085.571,599,1500000,7,0,6,1


0-无产品          1-Apple Pay       2-HCE       3-三星pay mst  4-三星pay ic  5-线上大商户          6-华为          7-小米          8-中兴          Y-主扫         Z-被扫

In [94]:
train.groupby('CARD_BRAND').size()

CARD_BRAND
1    312060
4       249
5       235
6        34
7        64
8         5
dtype: int64

# 结果文件

In [ ]:
a=train.loc[test['HCE_PROD_NM'] != '0'].encrypt_card_no.unique()
s=pd.Series(a)
s.to_csv('2.csv',index=False)

In [137]:
a=test.loc[test['HCE_PROD_NM'] != '0'].encrypt_card_no.unique()
pd.DataFrame(a).to_csv("1.csv",header=False,index=False)